In [ ]:
#크롤링시 필요한 라이브러리 불러오기
from bs4 import BeautifulSoup
import requests
import re
import datetime
from tqdm import tqdm
import sys

2. 크롤링 시 필요한 함수 만들기

In [ ]:
# 페이지 url 형식에 맞게 바꾸어 주는 함수 만들기
  #입력된 수를 1, 11, 21, 31 ...만들어 주는 함수
def makePgNum(num):
    if num == 1:
        return num
    elif num == 0:
        return num+1
    else:
        return num+9*(num-1)

#날짜 형식 변환해해 주는 함수 만들기
def chgDate(date):
  date2 = datetime.datetime.strptime(date,'%Y.%m.%d')
  date3 = datetime.datetime.strftime(date2,'%Y%m%d')
  return date3

# 크롤링할 url 생성하는 함수 만들기(검색어, 크롤링 시작 페이지, 크롤링 종료 페이지)
def makeUrl(search, start_pg, end_pg, start_dt, end_dt):
    if start_pg == end_pg:
        start_page = makePgNum(start_pg)
        s_date = chgDate(start_dt)
        e_date = chgDate(end_dt)
        url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + "&sort=1&photo=0&field=0&pd=3&ds=" + start_dt + "&de=" + end_dt + "&mynews=1&office_type=1&office_section_code=2&news_office_checked=1001&nso=so:dd,p:from" + s_date + "to" + e_date +",a:all&start=" + str(start_page)
        print("생성url: ", url)
        return url
    else:
        urls = []
        for i in range(start_pg, end_pg + 1):
            page = makePgNum(i)
            s_date = chgDate(start_dt)
            e_date = chgDate(end_dt)
            url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + "&sort=1&photo=0&field=0&pd=3&ds=" + start_dt + "&de=" + end_dt + "&mynews=1&office_type=1&office_section_code=2&news_office_checked=1001&nso=so:dd,p:from" + s_date + "to" + e_date +",a:all&start=" + str(page)
            urls.append(url)
        print("생성url: ", urls)
        return urls    

# html에서 원하는 속성 추출하는 함수 만들기 (기사, 추출하려는 속성값)
def news_attrs_crawler(articles,attrs):
    attrs_content=[]
    for i in articles:
        attrs_content.append(i.attrs[attrs])
    return attrs_content

# ConnectionError방지
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}

#html생성해서 기사크롤링하는 함수 만들기(url): 링크를 반환
def articles_crawler(url):
    #html 불러오기
    original_html = requests.get(i,headers=headers)
    html = BeautifulSoup(original_html.text, "html.parser")

    url_naver = html.select("div.group_news > ul.list_news > li div.news_area > div.news_info > div.info_group > a.info")
    url = news_attrs_crawler(url_naver,'href')
    return url

3. 크롤링할 네이버 뉴스 url추출하기

In [ ]:
#####뉴스크롤링 시작#####

#검색어 입력
search = input("검색할 키워드를 입력해주세요:")
#검색 시작할 페이지 입력
page = int(input("\n크롤링할 시작 페이지를 입력해주세요. ex)1(숫자만입력):")) # ex)1 =1페이지,2=2페이지... 
#검색 종료할 페이지 입력
page2 = int(input("\n크롤링할 종료 페이지를 입력해주세요. ex)1(숫자만입력):")) # ex)1 =1페이지,2=2페이지...
# 시작 날짜 페이지 입력
date = input("\n크롤링할 시작 날짜를 입력해주세요. ex)2021.01.01:")
# 종료 날짜 페이지 입력
date2 = input("\n크롤링할 종료 날짜를 입력해주세요. ex)2021.01.31:")

# naver url 생성
url = makeUrl(search,page,page2,date,date2)

#뉴스 크롤러 실행
news_titles = []
news_url =[] 
news_contents =[]
news_dates = []
news_press = []
for i in url:
    url = articles_crawler(url)
    news_url.append(url)


#제목, 링크, 내용 1차원 리스트로 꺼내는 함수 생성
def makeList(newlist, content):
    for i in content:
        for j in i:
            newlist.append(j)
    return newlist

    
#제목, 링크, 내용 담을 리스트 생성
news_url_1 = []

#1차원 리스트로 만들기(내용 제외)
makeList(news_url_1,news_url)

#NAVER 뉴스만 남기기
final_urls = []
for i in tqdm(range(len(news_url_1))):
    if "news.naver.com" in news_url_1[i]:
        final_urls.append(news_url_1[i])
    else:
        pass

검색할 키워드를 입력해주세요:"탄소중립"

크롤링할 시작 페이지를 입력해주세요. ex)1(숫자만입력):1

크롤링할 종료 페이지를 입력해주세요. ex)1(숫자만입력):281

크롤링할 시작 날짜를 입력해주세요. ex)2021.01.01:2021.07.01

크롤링할 종료 날짜를 입력해주세요. ex)2021.01.31:2021.12.31
생성url:  ['https://search.naver.com/search.naver?where=news&sm=tab_pge&query="탄소중립"&sort=1&photo=0&field=0&pd=3&ds=2021.07.01&de=2021.12.31&mynews=1&office_type=1&office_section_code=2&news_office_checked=1001&nso=so:dd,p:from20210701to20211231,a:all&start=1', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query="탄소중립"&sort=1&photo=0&field=0&pd=3&ds=2021.07.01&de=2021.12.31&mynews=1&office_type=1&office_section_code=2&news_office_checked=1001&nso=so:dd,p:from20210701to20211231,a:all&start=11', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query="탄소중립"&sort=1&photo=0&field=0&pd=3&ds=2021.07.01&de=2021.12.31&mynews=1&office_type=1&office_section_code=2&news_office_checked=1001&nso=so:dd,p:from20210701to20211231,a:all&start=21', 'https://search.naver.com/search.naver?where=ne

100%|██████████| 5600/5600 [00:00<00:00, 413770.61it/s]


4. 뉴스 본문 및 날짜 크롤링하기

In [ ]:
# 뉴스 내용 크롤링

for i in tqdm(final_urls):
    #각 기사 html get하기
    news = requests.get(i,headers=headers)
    news_html = BeautifulSoup(news.text,"html.parser")
    # 뉴스 제목 가져오기
    title = news_html.select_one("#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
    if title == None:
        title = news_html.select_one("#content > div.end_ct > div > h2")
    
    # 뉴스 본문 가져오기
    content = news_html.select("div#dic_area")
    if content == []:
        content = news_html.select("#articeBody")

    # 언론사 가져오기
    try:
      press = news_html.select_one(".media_end_head_top_logo > img")['title']
    except TypeError:
      press = news_html.select_one("#_CHANNEL_LAYER_366_0000868613 > span.media_end_head_top_channel_layer_text > strong")
    
    # 언론사 코드 가져오기
    #press_code = final_urls[39:41]

    # 기사 텍스트만 가져오기
    # list합치기
    content = ''.join(str(content))

    # html태그제거 및 텍스트 다듬기
    pattern1 = '<[^>]*>'
    title = re.sub(pattern=pattern1, repl='', string=str(title))
    content = re.sub(pattern=pattern1, repl='', string=content)
    press = re.sub(pattern=pattern1, repl='', string=str(press))
    pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
    content = content.replace(pattern2, '')

    news_titles.append(title)
    news_contents.append(content)
    news_press.append(press)
    
    #날짜 가져오기
    
    try:
        html_date = news_html.select_one("._ARTICLE_DATE_TIME")
        news_date = html_date.get_text('data-date-time')
    except AttributeError:
        news_date = news_html.select_one("#content > div.end_ct > div > div.article_info > span > em")
        news_date = re.sub(pattern=pattern1,repl='',string=str(news_date))
        
        

    news_dates.append(news_date)

print("검색된 기사 갯수: 총 ",(page2+1-page)*10,'개')
print("\n[뉴스 제목]")
print(news_titles)
print("\n[뉴스 링크]")
print(final_urls)
print("\n[뉴스 내용]")
print(news_contents)
print("\n[언론사]")
print(news_press)

print('news_title: ',len(news_titles))
print('news_url: ',len(final_urls))
print('news_contents: ',len(news_contents))
print('news_dates: ',len(news_dates))
print('news_press: ',len(news_press))

100%|██████████| 2800/2800 [23:04<00:00,  2.02it/s]IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)




5. 데이터 프레임 만들고 csv저장

In [ ]:

###데이터 프레임으로 만들기###
import pandas as pd

#데이터 프레임 만들기
news_df = pd.DataFrame({'date':news_dates,'press':news_press,'title':news_titles,'link':final_urls,'content':news_contents})
news_df

#중복 행 지우기
news_df = news_df.drop_duplicates(keep='first',ignore_index=True)
print("중복 제거 후 행 개수: ",len(news_df))

#데이터 프레임 저장
now = datetime.datetime.now() 
news_df.to_csv('{}_{}.csv'.format(search,now.strftime('%Y%m%d_%H시%M분%S초')),encoding='utf-8-sig',index=False)

중복 제거 후 행 개수:  2793
